# 转债轮动类策略构建与回测

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime

## 1 数据类型转换与过滤：日期转为 datetime.date 格式 ，并且对不符合条件的转债进行过滤

In [3]:
def convert_to_date(row:pd.Series) -> pd.Series:
    """
    将 datetime64[ns] 转换为 datetime.date 格式

    Args:
        - row (pd.Series): 一行元素，使用 pd.Series 进行表示

    Returns:
        pd.Series: 对这一行元素的格式进行转换
    """
    for idx in range(len(row)):
        if pd._libs.tslibs.timestamps.Timestamp == type(row[idx]):
            row[idx] = row[idx].date()
    return row

def credit_rating(ratings:pd.Series, threshold:str) -> pd.Series:
    """
    信用评级打分，查看信用是否 >= 阈值。我们规定：AAA > AA+ > AA > AA- > A+ > A > A-

    Args:
        - ratings (pd.Series): 输入一列评级，比如从 dataframe 中取出一列
        - threshold (str): 评级阈值，你希望筛选出评级 >= 多少 的可转债？
                         例如：threshold = AA- 代表筛选出评级至少是 AA- 级别的可转债

    Returns:
        pd.Series: 输出该列评级的每个元素是否 >= 阈值评级，如果 >= 阈值评级，返回 True ，否则返回 False
    """
    # 信用评价等级和分数的对应，信用评价等级越高，分数越高
    credit_to_score = {"AAA" : 10, "AA+" : 9,"AA" : 8,
        "AA-" : 7, "A+" : 6, "A" : 5, "A-" : 4}
    # 阈值分数
    threshold_score = credit_to_score.get(threshold, -1)
    # 如果没有在上面的 map 中找到该评级，就直接给 -1，最低信用分数
    rating_score = ratings.apply(lambda level : credit_to_score.get(level, -1))
    return rating_score >= threshold_score

def filter(trade_data:pd.DataFrame,
        remain_day_threshold:int, credit_rating_threshold:str, remain_scale_threshold:float) -> pd.DataFrame:
    """
    过滤不符合条件的可转债，主要过滤四类不符合条件的可转债 
    - 剔除公告强制赎回之后进行交易的情况，即当 交易日期 >= 强赎提示公告日 就剔除
    - 剔除在交易日剩余年限小于一定期限的可转债
    - 剔除评级过低的可转债
    - 剔除剩余规模过小的可转债 

    函数调用示例 filter(trade_data, 365, "AA-", 0.3)

    Args:
        - trade_data (pd.DataFrame): 从外部读入的周度交易数据，要求已经进行了日期处理 
        - remain_day_threshold (int): 剩余期限阈值，比如 365 天(一年)，希望转债剩余期限大于 365 天
        - credit_rating_threshold (str): 信用评级阈值，比如希望转债评级至少是 AA-
        - remain_scale_threshold (float): 剩余规模阈值(单位：亿元)，比如 0.3(表示3000万)表示希望剩余规模大于 0.3亿

    Returns:
        pd.DataFrame: 过滤后的数据，列数不变；只不过行数减少了，因为不符合要求的转债周度交易数据被删掉了
    """
    # NOTE 剔除公告强制赎回之后进行交易的情况，即当 交易日期 >= 强赎提示公告日 就剔除
    filtered_data = trade_data[pd.isna(trade_data["强赎提示公告日"]) | (trade_data["交易日期"] < trade_data["强赎提示公告日"])]
    # NOTE 剔除在交易日剩余年限小于一年的可转债，即当 到期日期 - 交易日期 <= 365 的情况下 就剔除
    filtered_data = filtered_data[(filtered_data["到期日期"] - filtered_data["交易日期"]). \
                                  apply(lambda datediff : datediff.days) > remain_day_threshold]
    # NOTE 我们保证评级在一定级别以上，比如 >= AA- 级
    filtered_data = filtered_data[credit_rating(filtered_data["发行时债项评级"], credit_rating_threshold)]
    # NOTE 筛选出转债剩余规模大于一定阈值的，比如 3000万(0.3亿)
    filtered_data = filtered_data[filtered_data["转债剩余规模(亿元)"] > remain_scale_threshold]
    return filtered_data.reset_index(drop = True)

## 2 实现轮动策略

In [4]:
class TradingDatabase:
    """ 交易数据库：用于查询交易相关的各种信息 """
    # 静态成员变量：交易数据库(以过滤后的数据进行交易)
    trade_database = None
    # 静态成员变量，表示所有可以交易的日期，从2017年初到2023年11月24日
    # 注意：日期是顺序排列的
    trade_date = None
    
    @staticmethod
    def set_database(filtered_data: pd.DataFrame):
        """
        设置周度交易数据库

        Args:
            - filtered_data (pd.DataFrame): 传入过滤后的转债交易周度数据
        """
        TradingDatabase.trade_database = filtered_data
        TradingDatabase.trade_date = np.unique(TradingDatabase.trade_database["交易日期"])

    @staticmethod
    def query_basic_info(codes) -> pd.DataFrame:
        """
        查询一支或多支转债的基本信息，债券WIND代码，债券简称，上市日期，到期日期，强赎提示公告日，发行时债项评级

        Args:
            codes option(str, list): 一支或多支转债 WIND代码，例如 110030.SH

        Returns:
            查询到的转债信息
        """
        basic_info = ["WIND代码", "债券简称", "上市日期", "到期日期", "强赎提示公告日", "发行时债项评级"]
        database = TradingDatabase.trade_database
        if type(codes) == str:
            return database[database["WIND代码"] == codes][basic_info].drop_duplicates()
        elif type(codes) == list:
            return database[database["WIND代码"].isin(codes)][basic_info].drop_duplicates()
        else:
            raise ValueError("输入类型不是字符串或者列表，无法查询")
    
    @staticmethod
    def convert_str_date(date:str) -> datetime.date:
        """
        将 "2017-01-04" 这样的字符串日期转化为 datetime.date 类型，并检查日期是否在每周收盘日列表内，如果没有，就报错

        Args:
            date (datetime.date): 输入的待查询日期

        Returns:
            类型转换后的日期
            
        Raises:
            ValueError
        """
        date = datetime.strptime(date, "%Y-%m-%d").date()
        if date not in TradingDatabase.trade_date:
            raise ValueError("输入的日期不在交易日范围内，交易日范围：2017~2023每周最后一个交易日")
        return date


    @staticmethod
    def query_trade_info(code:str, close_date:str, target:str = "日收盘价"):
        """
        查询一支转债在某一周(以该周最后一个交易日表示)的收盘价

        Args:
            - code (str): 转债 WIND代码
            - date (str): 该周的收盘日期，格式是字符串，例如 2017-01-02
            - target (str): 待查询指标名称，可以选择 日收盘价 或者 转股溢价率 或者 转债剩余规模(亿元)

        Returns:
            收盘价 / 转股溢价率 / 转债剩余规模(亿元)
        """
        database = TradingDatabase.trade_database
        close_date = TradingDatabase.convert_str_date(close_date)
        return database[(database["WIND代码"] == code) & (database["交易日期"] == close_date)][target].values[0]
    
    @staticmethod
    def get_next_date(date: str) -> datetime.date:
        """
        获取下一周的收盘日(下周的最后一个交易日)

        Args:
            - date (str): 本周最后一个交易日，格式是字符串，例如 "2017-01-06"

        Raises:
            ValueError: 当前日期已经是数据中的最后一日

        Returns:
            datetime.date: 下周最后一个交易日
        """
        date = TradingDatabase.convert_str_date(date)
        if date == TradingDatabase.trade_date[-1]:
            raise ValueError("当前日期已经是数据中的最后一日")
        return TradingDatabase.trade_date[np.where(TradingDatabase.trade_date == date)[0][0] + 1]

    @staticmethod
    def get_weekly_data(close_date:str):
        """
        获取本周(close_date所在周)收盘价、转股溢价率等信息

        Args:
            close_date (str): 该周的收盘日期，格式是字符串，例如 2017-01-02
        """
        database = TradingDatabase.trade_database
        close_date = TradingDatabase.convert_str_date(close_date)
        return database[database["交易日期"] == close_date].reset_index(drop = True)
    
    def weekly_data_filter(weekly_data1:pd.DataFrame, weekly_data2:pd.DataFrame) -> pd.DataFrame:
        """
        为了减少报错，防止出现：本周收盘日的数据中有交易信息的转债到了下周收盘日就没有了，这样
        就无法统计交易者下周持仓的总价值了，所以需要保证本周收盘日的转债在下周仍有交易数据，对于
        本周有交易数据而下周没有交易数据的转债，就需要过滤掉

        Args:
            - weekly_data1 (pd.DataFrame): 本周交易数据
            - weekly_data2 (pd.DataFrame): 下周交易数据(如果设定换仓频率是2，那就是weekly_data1后面第二周的交易数据)

        Returns:
            pd.DataFrame: 过滤掉本周有交易数据而下周没有交易数据的转债
        """
        weekly_data2 = weekly_data2[["WIND代码"]]
        return weekly_data1.merge(weekly_data2, on = "WIND代码", how = "inner")


In [17]:
class Account:
    """
    账户类：应该反映用户的初始资金，当前资金，持仓情况。
    可以添加或者删除转债持仓记录，也可以增加或者减少资金，但是二者的交互(比如买入转债)是由 AccountHandler 完成的
    """
    # 静态成员变量，表示所有可以交易的日期，从2017年初到2023年11月24日
    # 注意：日期是顺序排列的
    # trade_date = np.unique(trade_data["交易日期"])
    def __init__(self, init_money: float):
        """
        交易账户初始化，只需要设定初始存入的钱

        Args:
            init_money (float): 初始资金，单位是元，比如100万元则参数值是 1000000
        """
        if init_money < 0:
            raise ValueError("初始资金不能是负值")
        if init_money < 100000:
            raise ValueError("初始资金至少是10万")
        self.init_money = init_money # 还是需要记录初始资金的，为了方便计算单位净值
        self.account_money = self.init_money # account_money 表示 “账户资金”
        self.hold_shares = pd.DataFrame(columns = ["WIND代码", "持有数量"]) # 初始持仓
    def get_init_money(self) -> float:
        return self.init_money 
    def get_holdings(self) -> pd.DataFrame:
        """ 查看当前该账户的持仓 """
        return self.hold_shares
    def get_account_money(self) -> float:
        """ 查看当前账户资金 """
        return self.account_money
    
    def add_account_money(self, money: float):
        self.account_money += float(money)

    def sub_account_money(self, money: float):
        if money > self.account_money + 1:
            raise ValueError("账户金额扣除后是负值，故无法进行该操作")
        self.account_money -= float(money)

    def insert_record(self, code: str, amount: float):
        """
        插入新持仓(一次只能插入一条记录)

        Args:
            - code (_type_): 转债 WIND 代码
            - amount (_type_): 买入该转债的数量
        """
        self.hold_shares = pd.concat([self.hold_shares, 
                                      pd.DataFrame([[code, amount]], columns =  self.hold_shares.columns)], ignore_index = True)
    
    def in_holdings(self, code: str) -> bool:
        """
        查询当前账户是否持仓某个转债

        Args:
            - code (str): 转债 WIND 代码

        Returns:
            bool: 如果当前账户持仓该转债，就返回 True
        """
        return sum(self.hold_shares["WIND代码"] == code) > 0

    def update_amount(self, code: str, new_amount: float):
        """
        对 code 代码的转债增加/减少持仓数量

        Args:
            - code (str): 转债 WIND 代码

        Returns:
            bool: 如果当前账户持仓该转债，就返回 True
        """
        if not self.in_holdings(code):
            raise ValueError("没有持仓 " + code + ", 因此无法更新持仓量")
        if new_amount < 0:
            raise ValueError("持仓数量不能为负值")
        if new_amount == 0:
            self.remove_record(code)
            return
        self.hold_shares.loc[self.hold_shares["WIND代码"] == code, "持有数量"] = new_amount
          

    def query_holding_amount(self, code: str) -> float:
        """
        根据输入的 code 查询该 code 的持仓情况，如果该 code 没有持仓，就报错

        Args:
            - code (str): 转债 WIND 代码

        Returns:
            float: 该转债的持仓量
        """
        if not self.in_holdings(code):
            raise ValueError("没有持仓 " + code + ", 因此无法查询持仓量")
        return self.hold_shares.loc[self.hold_shares["WIND代码"] == code, "持有数量"].values

    def clear_holdings(self):
        """ 删除所有持仓记录，即创建新的空数据框 """
        self.hold_shares = pd.DataFrame(columns = ["WIND代码", "持有数量"])
        
    def remove_record(self, code:str):
        """
        删除某支转债的持仓记录

        Args:
            code (str): 转债 WIND 代码
        """
        if not self.in_holdings(code):
            raise ValueError("没有持仓 " + code + ", 因此无法删除该记录")
        self.hold_shares = self.hold_shares[~(self.hold_shares["WIND代码"] == code)].reset_index(drop = True)
    
    def get_holdings_value(self, close_date:str):
        """
        获取目前账户持仓价值 = (sum(本周收盘价 * 持仓数量)) 

        Args:
            close_date (str): 每周的收盘日
        """
        return sum([TradingDatabase.query_trade_info(code, close_date, "日收盘价") * amount\
                    for (code, amount) in list(self.hold_shares.values)]) 
    
    def get_account_value(self, close_date:str):
        """
        获取目前账户价值 = (sum(本周收盘价 * 持仓数量)) + 现金流的价值

        Args:
            close_date (str): 每周的收盘日
        """
        return self.get_holdings_value(close_date) + self.account_money

In [6]:
class Strategy:
    """用于实现轮动类策略周度数据选取可转债"""
    def __init__(self, weekly_data:pd.DataFrame, n:int):
        """
        Args:
            - weekly_data (pd.DataFrame): 一周的可转债交易数据
            - n(int): 每个策略选几只股票(如果一周的标的 <= n，则本周标的会被全部选择)
        """
        self.weekly_data = weekly_data
        self.select_num = min(len(weekly_data), n)
        self.strategy_idx_map = {
            0 : "双低策略",
            1 : "低溢价率策略",
            2 : "高信用评级策略",
            3 : "小规模策略"
        }

    def get_strategy_name(self, strategy_idx:int):
        """ 根据 strategy_idx 获取该策略的具体名称 """
        return self.strategy_idx_map.get(strategy_idx, "输入的策略代码无效")

    def do_strategy(self, strategy_idx:int) -> pd.DataFrame:
        """
        根据输入的 strategy_idx 决定执行何种轮动策略
        Args:
            strategy_idx (int): 策略代码 0——双低策略 1——低溢价率策略 2——高信用评级策略 3——小规模策略

        Returns:
            pd.DataFrame: 筛选出的 n 支股票
        """
        if strategy_idx == 0:
            return self.double_low()
        elif strategy_idx == 1:
            return self.low_premium_rate()
        elif strategy_idx == 2:
            return self.high_credit()
        elif strategy_idx == 3:
            return self.low_scale()
        else:
            raise ValueError("输入的轮动类策略代码没有匹配的策略")
        
    def double_low(self) -> pd.DataFrame: 
        """
        使用双低策略对某周的数据进行选股；双低 = 日收盘价 + 转股溢价率 * 100

        Returns:
            pd.DataFrame: 筛选出的 n 支股票
        """
        self.weekly_data["双低"] = self.weekly_data["日收盘价"] + self.weekly_data["转股溢价率"] * 100
        return self.weekly_data.sort_values(by = "双低").iloc[:self.select_num]

    def low_premium_rate(self) -> pd.DataFrame:
        """
        低溢价率策略，选出转股溢价率最低的前10支股票

        Returns:
            pd.DataFrame: 筛选出的 n 支股票
        """
        return self.weekly_data.sort_values(by = "转股溢价率").iloc[:self.select_num]
    
    def high_credit(self) -> pd.DataFrame:
        """
        高信用评级(>= AA) + 双低；此策略的构建需要在 filter 方法中设置转债评级为 AA(或者更高)

        Returns:
            pd.DataFrame: 筛选出的 n 支股票
        """
        credit_levels = np.unique(TradingDatabase.trade_database["发行时债项评级"])
        if "AA-" in credit_levels:
            raise ValueError("高信用评级策略要求待选转债的评级至少为AA级")
        return self.double_low()
    
    def low_scale(self) -> pd.DataFrame:
        """
        小剩余规模策略，选择转债剩余规模最小的 n 支转债(注意：前面的filter还会过滤掉一些市值规模过小的转债)

        Returns:
            pd.DataFrame: 筛选出的 n 支股票
        """
        return self.weekly_data.sort_values(by = "转债剩余规模(亿元)").iloc[:self.select_num]

In [25]:
class AccountHandler:
    """
    用于周度/双周/月度进行换仓操作，也可以 建仓 / 平仓
    不考虑交易费用，采用简化思路进行换仓：即每周最后一个交易日卖出所有持仓(清空持仓)，再买入新持仓
    """
    def __init__(self, init_money: float, n: int, change_freq: int, fee_rate: float,
                 begin_date: str, end_date: str, strategy_idx: int):
        """
        创建账户操作对象

        Args:
            - init_money (float): 设置初始资金
            - n (int): 望每次持仓几支可转债？比如周度策略，每周持有 10 支转债，然后轮动
            - change_freq (int):  换仓频率，单位是周，1 2(2周，半个月) 4(4周，一个月)
            - fee_rate (float): 交易费用率，例如：每次交易的费用率是 0.024% 即 0.00024
            - begin_date (str): 建仓日期，格式是字符串"2017-01-06"，必须是2017~2023每周收盘日其中之一
            - end_date (str): 平仓日期，格式是字符串"2023-11-24"，必须是2017~2023每周收盘日其中之一
            - strategy_idx (int): 策略代码 0——双低策略 1——低溢价率策略 2——高信用评级策略 3——小规模策略

        Raises:
            ValueError: 传入的参数值不符合上述说明，直接报错，禁止创建对象
        """
        self.account = Account(init_money)
        self.holding_num = n # 希望每次持仓几支可转债？比如 10 支，注意，部分周的可选标的可能小于 n
        self.change_freq = change_freq # 换仓频率，单位是周，1 2 4
        self.fee_rate = fee_rate # 交易费用率，例如：每次交易的费用率是 0.024% 即 0.00024
        if self.change_freq not in [1, 2, 4]:
            raise ValueError("换仓频率 change_freq 必须是 1 / 2 / 4中的一个")
        self.begin_date = begin_date # 建仓日期
        self.end_date = end_date # 平仓日期
        self.strategy_idx = strategy_idx
        if self.end_date <= self.begin_date:
            raise ValueError("平仓日期必须在建仓日期之后")
        self.trade_dates = self.set_trade_dates()  # 根据用户配置，设定换仓日期列表(注意：返回的是字符串格式日期)
        
    def select(self, date: str) -> pd.DataFrame:
        """
        根据给定策略进行选股，注意，为了防止出现部分转债
        本期有收盘价，但是由于前面的过滤或者转债到期导致下期没有收盘价，
        需要保证在获取本周转债交易数据时，该转债在下一期也需要有交易数据

        Args:
            - date (str): 具体选债的日期(轮动/建仓/平仓日期)
        
        Returns:
            选出的 n 支转债(如果当周可选标的数量 x <= n，则选出 x 支转债)
        """
        trade_days = self.trade_dates
        weekly_data = TradingDatabase.get_weekly_data(date)
        # 如果不是最后一个交易日
        if date != trade_days[-1]:
            next_weekly_data = TradingDatabase.get_weekly_data(trade_days[trade_days.index(date) + 1])
            weekly_data = TradingDatabase.weekly_data_filter(weekly_data, next_weekly_data)
        return Strategy(weekly_data, self.holding_num).do_strategy(self.strategy_idx)

    def buy_one_bond(self, wind_code: str, buy_amount: float, date: str):
        """
        购买一只转债：需要进行两个方面的操作：①插入持仓记录；②账户金额减少

        Args:
            - wind_code (str): 买入转债的 wind 代码
            - buy_amount (float): 买入数量
            - date (str): 交易日期(只能是每一周的收盘日)
        """
        self.account.insert_record(wind_code, buy_amount)
        # 减少的费用 = 购买费用 + 交易费用
        self.account.sub_account_money(
            TradingDatabase.query_trade_info(wind_code, date) * buy_amount * (1 + self.fee_rate))
    
    def add_holding(self, wind_code: str, add_amount: float, date: str):
        """
        增加既有持仓转债的持仓量：①更新持仓量；②账户金额减少

        Args:
            - wind_code (str): 买入转债的 wind 代码
            - buy_amount (float): 增加持仓的数量
            - date (str): 交易日期(只能是每一周的收盘日)
        """
        current_holding_amount = self.account.query_holding_amount(wind_code)
        self.account.update_amount(wind_code, current_holding_amount + add_amount)
        # 减少的费用 = 购买费用 + 交易费用
        self.account.sub_account_money(
            TradingDatabase.query_trade_info(wind_code, date) * add_amount * (1 + self.fee_rate))        

    def sell_one_holding(self, wind_code: str, date: str):
        """
        卖出一只持仓转债：需要进行两个方面的操作：①删除持仓记录；②账户金额增加

        Args:
            - wind_code (str): 希望卖出转债的 wind 代码
            - date (str): 交易日期(只能是每一周的收盘日)
        """
        sell_amount = self.account.query_holding_amount(wind_code)
        self.account.remove_record(wind_code)
        # 增加的费用 = 卖出费用 - 交易费用
        self.account.add_account_money(
            TradingDatabase.query_trade_info(wind_code, date) * 
            sell_amount * (1 - self.fee_rate))

    def buy_selected_bond(self, date:str):
        """
        使用均分法，买入选中的全部转债标的，用于建仓

        Args:
            - date (str): 交易日期，必须是2017~2023每周收盘日其中之一
        """
        selected_bond = self.select(date)
        # 表示为每只转债分配的账户资金[平均分配]
        per_bond_money = self.account.get_account_money() / len(selected_bond)
        selected_bond["买入数量"] = per_bond_money / (selected_bond["日收盘价"] * (1 + self.fee_rate))
        for bond in selected_bond[["WIND代码", "买入数量"]].values: 
            # 购买一只转债，增加持仓记录，减少账户现金
            self.buy_one_bond(bond[0], bond[1], date)

    def sell_all_holdings(self, date: str) -> float:
        """
        卖出全部持仓，结算成现金，用于平仓

        Args:
            - date (str): 交易日期，必须是2017~2023每周收盘日其中之一

        Returns:
            float: 将当前持仓全部出售后，账户总价值(持仓价值 + 现金价值)
        """
        self.account.add_account_money((1 - self.fee_rate) * self.account.get_holdings_value(date))
        self.account.clear_holdings()

    def construct(self):
        """ 
        建仓操作，在设定的 begin_date 日建仓，建仓规则：均分初始资金。
        比如 1000000 资金，购买10支转债，每支分配 100000 
        """
        self.buy_selected_bond(self.begin_date)

    def destroy(self):
        """
        平仓操作，卖出持有的所有可转债，注意只能在 end_date 平仓

        Returns:
            float: 平仓后得到的金额
        """
        self.sell_all_holdings(self.end_date)
    
    def get_sell_list(self, selected_bonds: pd.DataFrame) -> list:
        """
        根据账户当前持仓和本周新筛选的转债，考察哪些持仓股票需要卖出

        Args:
            - selected_bonds (pd.DataFrame): 本周筛选出的转债，把函数select的返回结果传入即可

        Returns:
            list: 需要卖出现有持仓转债的wind代码列表
        """
        current_holdings = self.account.get_holdings()
        return list(set(selected_bonds["WIND代码"]).union(set(current_holdings["WIND代码"])) \
               - set(selected_bonds["WIND代码"]))

    def get_buy_list(self, selected_bonds: pd.DataFrame) -> list:
        """
        根据账户当前持仓和本周新筛选的转债，考察哪些股票新加入了排名，需要买入

        Args:
            - selected_bonds (pd.DataFrame): 本周筛选出的转债，把函数select的返回结果传入即可

        Returns:
            list: 需要卖出现有持仓转债的wind代码列表
        """
        current_holdings = self.account.get_holdings()
        return list(set(selected_bonds["WIND代码"]).union(set(current_holdings["WIND代码"])) \
               - set(current_holdings["WIND代码"]))

    def cal_amount(self, wind_code: str, per_bond_money: float, date: str) -> float:
        """
        在设定的交易费率和特定交易日的情况下，给定购买某只转债的预算金额 per_bond_money
        返回能够购买的转债数量(cal是calculate的缩写)

        Args:
            - wind_code (str):  买入转债的 wind 代码
            - per_bond_money (float): 买入这个转债希望花费的金额
            - date (str): 交易日期，必须是每周的收盘日(该说法等价于：每周最后一个交易日)

        Returns:
            float: 购买的数量
        """
        return per_bond_money/((1 + self.fee_rate) * TradingDatabase.query_trade_info(wind_code, date))

    def change(self, date: str):
        """
        换仓操作：每周先卖出掉出排名的转债，再买入新进入排名的转债，[尽量]保证每支转债当周持仓的金额是均匀分布的；
        
        细节处理举例: \n
        1. 如果有2支转债掉出排名(持仓)，有2支转债新进入排名，则卖出2支持仓且掉出排名的转债，资金均分买入2支持仓转债 \n
        2. 如果有3支转债掉出排名(持仓)，由于某周转债标的较少，只有2支转债新进入排名，则卖出3支持仓且掉出排名的转债，
           得到的资金均匀分配买入新进入排名的2支转债 \n
        3. 如果有2支转债掉出排名(持仓)，没有转债进入排名，则卖出2支持仓且掉出排名的转债，得到的资金均分给原有持仓加仓 \n
        4. 如果有2支转债掉出排名，3支转债新进入排名，则卖出2支持仓且掉出排名的转债，得到的资金均分买入3支持仓转债 \n

        Args:
            - date (str): 交易日期，必须是2017~2023每周收盘日其中之一

        Returns:
            float: 上周持有n支转债得到的总价值
        """
        # 获取本周在某种策略下排名前 N 的转债
        selected_bond = self.select(date)
        sell_list = self.get_sell_list(selected_bond)
        buy_list = self.get_buy_list(selected_bond)
        # 卖出 sell_list 中的转债
        for bond_code in sell_list:
            self.sell_one_holding(bond_code, date)
        # 如果根本没有卖出持仓，那就没有后续的换仓交易了
        if len(sell_list) == 0:
            return
        # 资金过低，就不再交易了
        if self.account.get_account_money() < 1000:     
            return
        # 根据情况讨论具体轮动方式
        if len(buy_list) > 0: 
            # 情况1：卖出了k支，又买入t支(t > 0)，则卖出k支得到的资金均分到 t 支新进入转债上
            # per_bond_money 为买入的 k 支转债均匀分配资金
            per_bond_money = self.account.get_account_money() / len(buy_list)
            for bond_code in buy_list:
                # 买入数量 = 分配金额 / [收盘价 * (1 + 交易费率)]
                buy_amount = self.cal_amount(bond_code, per_bond_money, date)
                self.buy_one_bond(bond_code, buy_amount, date)
        else: # 情况2：买入0支，此时为卖出后剩余持仓均匀分配金额加仓
            per_bond_money = self.account.get_account_money() / len(self.account.get_holdings())
            for bond_code in self.account.get_holdings()["WIND代码"]:
                # 加仓数量 = 分配金额 / [收盘价 * (1 + 交易费率)]
                add_amount = self.cal_amount(bond_code, per_bond_money, date)
                self.add_holding(bond_code, add_amount, date)
    
    def set_trade_dates(self) -> list:
        """根据设置的交易频率，以及开始交易日和最后交易日，设定换仓日期列表(注意：返回的是字符串格式日期)"""
        trade_date = TradingDatabase.trade_date
        begin_date = TradingDatabase.convert_str_date(self.begin_date)
        end_date = TradingDatabase.convert_str_date(self.end_date)
        selected_date = trade_date[(trade_date >= begin_date) & (trade_date <= end_date)]
        selected_date = list(selected_date[::self.change_freq])
        if end_date not in selected_date:
            selected_date.append(end_date)
        return [datetime.strftime(elem, "%Y-%m-%d") for elem in selected_date]
    
    def get_trade_dates(self):
        """ 获取成员变量：交易日期列表 """
        return self.trade_dates

    def back_testing(self):
        """根据设定的参数，对策略进行回测，计算净值"""
        # 用这个列表统计账户的总价值走势
        account_value_list = [self.account.get_init_money()]
        # 获取根据用户参数设定下的交易日列表
        trade_dates = self.trade_dates
        print(trade_dates[0],"1")
        # 建仓
        self.construct()
        # 根据用户设定的频率换仓
        for idx in range(1, len(trade_dates) - 1):
            self.change(trade_dates[idx])
            account_value_list.append(self.account.get_account_value(trade_dates[idx]))
            print(trade_dates[idx], " 持仓数：", len(self.account.get_holdings()), self.account.get_account_value(trade_dates[idx]) / self.account.get_init_money())
        # 最后平仓
        self.destroy()
        account_value_list.append(self.account.get_account_value(trade_dates[-1]))
        print(trade_dates[-1], self.account.get_account_value(trade_dates[-1]) / self.account.get_init_money())
        return np.array(account_value_list) / self.account.get_init_money()

In [8]:
# 根据净值数据计算区间最大回撤等指标，还可以绘制净值走势
class PerformanceEvaluate:
    def __init__(self, net_value:np.ndarray, trade_dates:list):
        """
        评估某策略得到的累计净值走势相关的各种评价指标

        Args:
            - net_value (np.ndarray): 净值数据，根据 AccountHandler 类的对象得到
            - trade_dates (list): 交易日期列表，注意列表中的日期格式是字符串，例如 "2023-11-24"

        Raises:
            ValueError: 如果你的净值数据和交易日期不匹配，就抛出异常
        """
        self.net_value = net_value
        self.trade_dates = self.init_convert(trade_dates)
        if len(self.net_value) != len(self.trade_dates):
            raise ValueError("净值数据和交易日期两个列表长度不一致，无法进行日期与净值数据的匹配")
        
    def init_convert(self, trade_dates):
        """ 进行字符串格式日期 到 datetime.date 的转换 """
        return [datetime.strptime(elem, "%Y-%m-%d").date()  for elem in trade_dates]
    def cal_drawback(self) -> np.ndarray:
        """ 计算净值数据对应的回撤 """
        drawback = []
        current_max = self.net_value[0]
        for idx in range(len(self.net_value)):
            current_max = max(current_max, self.net_value[idx])
            drawback.append(self.net_value[idx]/current_max - 1)
        return np.array(drawback)
    
    def get_annual_rate(self):
        """ 计算年化收益率 """
        return 0

    def cal_sharpe_ratio(self, risk_free_rate = 0.015):
        """
        计算夏普比 = (年化收益率 - R_f(无风险利率)) / 年化波动率

        Args:
            risk_free_rate (float, optional): 无风险利率，默认是 0.015.
        """
        return 0

## 脚本代码

In [9]:
# 读入数据
trade_data = pd.read_excel("可转债周度交易数据.xlsx")
# 日期格式强转，以免发生类型错误
trade_data['上市日期'] = trade_data['上市日期'].astype(object)
trade_data['到期日期'] = trade_data['到期日期'].astype(object)
trade_data['强赎提示公告日'] = trade_data['强赎提示公告日'].astype(object)
# NOTE 这里所有的日期必须转换为 datetime.date 格式，否则后续在进行时间处理时一定会报错
trade_data[['上市日期', '到期日期', '强赎提示公告日']] = \
    trade_data[['上市日期', '到期日期', '强赎提示公告日']]. \
    apply(convert_to_date, axis = 1)
trade_data["交易日期"] = trade_data["交易日期"].apply(lambda date_str : datetime.strptime(date_str, "%Y-%m-%d").date())
trade_data["债券简称"] = trade_data["债券简称"].apply(lambda bond_name : bond_name.replace("(退市)", ""))
trade_data

,债券代码,债券简称,上市日期,到期日期,强赎提示公告日,发行时债项评级,WIND代码,交易日期,日收盘价,转股溢价率,转债剩余规模(亿元)
0,110030,格力转债,2015-01-13,2019-12-25,NaT,AA,110030.SH,2017-01-06,114.860,0.413805,9.780730
1,110031,航信转债,2015-06-30,2021-06-12,NaT,AAA,110031.SH,2017-01-06,108.650,1.285426,23.995080
2,110032,三一转债,2016-01-18,2022-01-04,2019-03-04,AA+,110032.SH,2017-01-06,111.910,0.285638,44.998170
3,110033,国贸转债,2016-01-19,2022-01-05,2021-09-25,AA+,110033.SH,2017-01-06,115.720,0.240584,27.996640
4,110034,九州转债,2016-01-29,2022-01-15,NaT,AA+,110034.SH,2017-01-06,124.920,0.148333,14.996850
...,...,...,...,...,...,...,...,...,...,...,...
89047,128138,侨银转债,2020-12-24,2026-11-17,NaN,AA-,128138.SZ,2023-11-24,117.214,1.638043,4.198741
89048,128141,旺能转债,2021-01-18,2026-12-17,NaN,AA,128141.SZ,2023-11-24,120.030,0.203270,12.701445
89049,128142,新乳转债,2021-01-19,2026-12-18,NaN,AA,128142.SZ,2023-11-24,111.300,0.603511,7.179705
89050,128143,锋龙转债,2021-01-29,2027-01-08,NaN,A+,128143.SZ,2023-11-24,137.588,0.211807,1.734365


In [15]:
# 获得过滤后的数据
filtered_data = filter(trade_data, 365, "AA-", 1)
# 初始化周度交易数据库类，该类提供了多种接口，便于查询交易数据
TradingDatabase.set_database(filtered_data)
# 创建账户处理实例，用于运行策略
account_handler = AccountHandler(100000, 10, 4, 0.024 / 100, "2023-01-06", "2023-11-24", 0)
# 对策略进行回测，获取净值数据
netval_list = account_handler.back_testing()
netval_list

2023-01-06 1


TypeError: get_account_value() missing 1 required positional argument: 'close_date'

In [64]:
for elem in pd.Series([1,2,3]):
    print(elem)

1
2
3


In [71]:
set(pd.Series([1,2,3,1])).union(set({1, 2, 4, 5})) - set(pd.Series([1,2,3,1]))

{4, 5}